In [1]:
!export TOKENIZERS_PARALLELISM=false
!python -m pip install --upgrade pip
!pip install typing_extensions pydantic openai
!pip install datasets transformers peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.9 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 445.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 210.6 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 309.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 175.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 261.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 273.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 159.9 MB/s eta 0:00:00
   ━━━━━

In [2]:
# 학습 모델 로드 및 토큰 호출 한번더
from peft import PeftModel
from transformers import AutoModelForCausalLM,AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained("google/gemma-3-12b-it", torch_dtype="auto", device_map="cuda")
model = PeftModel.from_pretrained(base_model, "ohdyo/trip_qa_gem13b_v2")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-12b-it",add_bos=True)

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/109k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [10]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
!pip install flask --ignore-installed blinker

In [4]:
from flask import Flask, request, jsonify
app = Flask(__name__)

In [13]:
chat_history = []

# 프롬프트 생성 함수
def make_prompt(history, new_question):
    system_prompt = """너는 항공기 반입 금지 물품 및 여행 정보를 전문적으로 알려주는 비서야.
답변이 없는 질문에 대해서만 답변해줘 설명도 달아줘. 다른 추가 질문은 하지 마.\n\n"""
    
    history_prompt = ""
    for qa in history:
        history_prompt += f"### 질문:\n{qa['question']}\n\n### 답변:\n{qa['answer']}\n\n"
    
    history_prompt += f"### 질문:\n{new_question}\n\n### 답변:"
    return system_prompt + history_prompt

# 모델 질의 함수
def ask_with_generate(new_question, max_new_tokens=300):
    prompt = make_prompt(chat_history,new_question)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = generated_text.replace(prompt, "").strip()

    chat_history.append({"question": new_question, "answer": answer})
    
    return answer

In [6]:
@app.route('/input', methods=['POST'])
def handle_input():
    try:
        data = request.get_json()
        question = data.get("question", "")
        if not question:
            return jsonify({"error": "질문이 비어있습니다."}), 400

        result = ask_with_generate(new_question=question)
        return jsonify({"question": question, "answer": result})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [ ]:
app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.16.16.2:8080
Press CTRL+C to quit
